In [35]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
import joblib

In [2]:
# There are two classes of interests among the images in dataset
cls_of_int = ['Covid', 'Normal']

In [11]:
basePath = "Data"
trainPath = os.path.join(basePath,"train")
testPath = os.path.join(basePath,"test")
validPath = os.path.join(basePath,"valid")

trainCovid = os.path.join(trainPath,"Covid")
trainNormal = os.path.join(trainPath,"Normal")

testCovid = os.path.join(testPath,"Covid")
testNormal = os.path.join(testPath,"Normal")

validCovid = os.path.join(basePath,"valid","Covid")
validNormal = os.path.join(basePath,"valid","Normal")

In [3]:
# using a data generator for data augmentation
train_data_generator = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    vertical_flip = False,
    fill_mode='nearest',
    featurewise_center = True,
    featurewise_std_normalization = True)

In [4]:
valid_data_generator = ImageDataGenerator(rescale = 1./255)

In [5]:
test_data_generator = ImageDataGenerator(rescale = 1./255)

In [9]:
train_generator = train_data_generator.flow_from_directory(
    directory=trainPath,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 11047 images belonging to 2 classes.


In [13]:
valid_generator = valid_data_generator.flow_from_directory(
    directory= validPath,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 2208 images belonging to 2 classes.


In [15]:
test_generator = test_data_generator.flow_from_directory(
    directory= testPath,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=11,
    class_mode=None,
    shuffle=False,
    seed=42
)

Found 2761 images belonging to 2 classes.


In [18]:
pretrained_model = tf.keras.applications.InceptionV3(
        weights='imagenet',
        include_top=False ,
        input_shape=(299,299,3)
    )
pretrained_model.trainable = False
    
model = tf.keras.Sequential([ 
        pretrained_model,   
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

87916544/87910968 [==============================] - 3s 0us/step


In [19]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer = opt ,
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 8, 8, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 21,806,882
Trainable params: 4,098
Non-trainable params: 21,802,784
_________________________________________________________________


In [21]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
345/345 [==============================] - 500s 1s/step - loss: 0.3683 - accuracy: 0.8422 - val_loss: 0.3258 - val_accuracy: 0.8582
Epoch 2/10
345/345 [==============================] - 510s 1s/step - loss: 0.3035 - accuracy: 0.8788 - val_loss: 0.2793 - val_accuracy: 0.8877
Epoch 3/10
345/345 [==============================] - 509s 1s/step - loss: 0.2852 - accuracy: 0.8872 - val_loss: 0.2798 - val_accuracy: 0.8940
Epoch 4/10
345/345 [==============================] - 506s 1s/step - loss: 0.2819 - accuracy: 0.8892 - val_loss: 0.2791 - val_accuracy: 0.8872
Epoch 5/10
345/345 [==============================] - 510s 1s/step - loss: 0.2757 - accuracy: 0.8904 - val_loss: 0.3856 - val_accuracy: 0.8533
Epoch 6/10
345/345 [==============================] - 536s 2s/step - loss: 0.2643 - accuracy: 0.8937 - val_loss: 0.2800 - val_accuracy: 0.8863
Epoch 7/10
345/345 [==============================] - 526s 2s/step

In [25]:
model.evaluate(valid_generator,steps=STEP_SIZE_VALID)

69/69 [==============================] - 75s 1s/step - loss: 0.2716 - accuracy: 0.8913


[0.2716220021247864, 0.8913043737411499]

In [27]:
model.evaluate(test_generator)

251/251 [==============================] - 12s 48ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


[0.0, 0.0]

In [28]:
score = model.evaluate(valid_generator,batch_size=32)
print("Accuracy: {:.2f}%".format(score[1] * 100)) 
print("Loss: ",score[0])

69/69 [==============================] - 70s 1s/step - loss: 0.2716 - accuracy: 0.8913
Accuracy: 89.13%
Loss:  0.2716220021247864


In [33]:
score = model.evaluate(test_generator,batch_size=11)
print("Accuracy: {:.2f}%".format(score[1] * 100)) 
print("Loss: ",score[0])

251/251 [==============================] - 12s 47ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Accuracy: 0.00%
Loss:  0.0


In [38]:
model.save('imagegeneratormodel')

INFO:tensorflow:Assets written to: imagegeneratormodel\assets
